# Scapy

Scapy je python program, ki omogoča sniff-ing, disekcijo, kreiranje in pošiljanje paketkov.

## Installing

https://scapy.readthedocs.io/en/latest/installation.html#installing-scapy-v2-x

$ pip install scapy

Ko imamo inštaliran Scapy ga lahko uporabimo v python skriptah oziroma zaženemo v terminalu, kar nam odpre interaktivni terminal. Če želimo tudi pošiljati paketke moramo zagnati terminal z sudo privilegiji.

$ scapy

$ sudo scapy

---

Za začetek lahko preverimo kateri interface bo scapy po default uporabil:

```
>>> $ conf.iface
```

Če želimo uporabiti drugega lahko to spremenimo z ukazom:
```
>>> conf.iface="en3"
```

---

**Permission error** oziroma **No module named 'scapy'**.
>The PYTHON sys.path variable was different between regular and SUDO use (and SUDO is required for scapy module).
There were a couple different approaches I found to solve it: This ended up solving it with the .bashrc suggestion. Essentially, the python module paths needed to be added to the sudo user or environment. Good luck to anybody else who has this issue!

* https://askubuntu.com/questions/1243933/scapy-module-not-found-when-running-python-script-with-sudo
* https://askubuntu.com/questions/57915/environment-variables-when-run-with-sudo/713137#713137
* https://stackoverflow.com/questions/7969540/pythonpath-not-working-for-sudo-on-gnu-linux-works-for-root

Kar je men začel delat je:
```
$ sudo -E env PATH=$PATH PYTHONPATH=$PYTHONPATH scapy
```

## Sniffing - from terminal

Začnimo s preprostim sniffingom. V terminalu to dosežemo s sledečimi ukazi:

```
>>> pkt = sniff(count=1)

>>> type(pkt)

>>> pkt

>>> pkt[0].summary()

```

V našem primeru preprosto sniff-amo 1 paketek. Funkcija nam vrne list sniffanih paketkov in z **summary()** metodo dobimo okviren opis paketka.

Oziroma, če želimo zajeti več paketkov naenkrat in izpisati okvirne informacije:

```
>>> pkts = sniff(count=10)
>>> pkts.summary()
```

Scapy paketke prikazuje v nested dictionary strukturi, kjer je vsaka višja plast dictionary znotraj prejšnje plasti. Vsako polje, kot naprimer **IP dst**, je sestavljeno iz **key:value** para. Vrednosti so mutable kar pomeni, da jih lahko poljubno prepišemo.

---

Z metodo **sumary()** dobimo le okvirne informacije. Če želimo bolj natančen izpis paketka lahko uporabimo **show()** metodo:

```
>>> pkt[0].show()
```

Premikanje po paketku je čisto preprosto:

```
>>> pkt[0][IPv6]
>>> pkt[0]["IPv6"]
>>> pkt[0][IPv6].summary()
>>> pkt[0][IPv6].show()
```

Če želimo videti kako bi izgledala syntaxa, da ustvarimo ta specifični paketek, lahko uporabimo **command()** metodo.

```
>>> pkts[0].command()
```

Da pridobimo specifično vrednost posameznega layerja:

```
>>> pkt[0][IPv6].src
```

---

Znotraj scapy terminala lahko tudi pišemo python kodo:

```
>>> pkts = sniff(count=20)
>>> for pkt in pkts:
...:    if pkt.haslayer(IPv6):
...:        print(pkt[IPv6].src)
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910
2600:1901:1:c36::
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910
2600:1901:1:c36::
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910
2600:1901:1:c36::
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910
2a03:2880:f007:1:face:b00c:0:1
2a03:2880:f007:1:face:b00c:0:1
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910
2a03:2880:f007:1:face:b00c:0:1
2a03:2880:f007:1:face:b00c:0:1
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910
2a03:2880:f007:1:face:b00c:0:1
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910
2a00:1a20:2900:140a:8abc:7bd8:2ee3:7910           
```

Malo bolj natančno si poglejmo **sniff()** metodo.

* **count** - povemo koliko paktekov naj zajeme. Če nastavimo vrednost 0 potem konstantno spremljamo promet
* **iface** - pove kateri interface naj spremlja. Če tega ne nastavimo bo vzel privzeti interfrace
* **prn** - funkcija katero naj izvede ob vsakem zajemu novega paketka. Primer: lambda x: x.summary()
* **store** - nam pove ali naj prejete paketke shranimo ali ne. V kolikor jih ne želimo shraniti nastavimo na 0 
* **timeout** - pove koliko sekund naj spremlja promet
* **filter** - sprejme BPF Syntax s katero lahko filtriramo prejete paketke... https://biot.com/capstats/bpf.html


## Building a packet

Za primer vzemimo gradnjo ARP paketka 
* https://www.javatpoint.com/arp-packet-format
* https://www.techtarget.com/searchnetworking/definition/Address-Resolution-Protocol-ARP
> When a new computer joins a LAN, it is assigned a unique IP address to use for identification and communication. When an incoming packet destined for a host machine on a particular LAN arrives at a gateway, the gateway asks the ARP program to find a MAC address that matches the IP address. A table called the ARP cache maintains a record of each IP address and its corresponding MAC address.



Za pomoč preprosto prvo zajamimo ARP paketek in nato z metodo **command()** izpišimo kako zgleda izgradnja paketka.

```
>>> pkts = sniff(count=5, filter="arp", prn=lambda x: x.summary())

Ether / ARP who has 192.168.8.145 says 192.168.8.1
Ether / ARP is at 80:30:49:e5:6d:c5 says 192.168.8.145
Ether / ARP who has 192.168.8.147 says 192.168.8.1 / Padding
Ether / ARP who has 192.168.8.145 says 192.168.8.1
Ether / ARP is at 80:30:49:e5:6d:c5 says 192.168.8.145
```

S pomočjo **show()** komande vidimo kako je zgrajen paketek. S pomočjo **command()** metode vidimo, kako izgleda syntaxa, da zgradimo tak paketek.

```
>>> pkts[0].show()
###[ Ethernet ]### 
  dst       = 80:30:49:e5:6d:c5
  src       = ac:92:32:e8:33:f5
  type      = ARP
###[ ARP ]### 
     hwtype    = 0x1
     ptype     = IPv4
     hwlen     = 6
     plen      = 4
     op        = who-has
     hwsrc     = ac:92:32:e8:33:f5
     psrc      = 192.168.8.1
     hwdst     = 00:00:00:00:00:00
     pdst      = 192.168.8.145

>>> pkts[0].command()
"Ether(dst='80:30:49:e5:6d:c5', src='ac:92:32:e8:33:f5', type=2054)/ARP(hwtype=1, ptype=2048, hwlen=6, plen=4, op=1, hwsrc='ac:92:32:e8:33:f5', psrc='192.168.8.1', hwdst='00:00:00:00:00:00', pdst='192.168.8.145')"
```

Kakor vidimo je ARP paketek sestavljen iz dveh plasti. S pomočjo funkcije **ls(LAYER)** lahko vidimo, katera polja potrebujemo izpolniti za kateri layer.

```
>>> ls(Ether)
dst        : DestMACField                        = ('None')
src        : SourceMACField                      = ('None')
type       : XShortEnumField                     = ('36864')
>>> ls(ARP)
hwtype     : XShortField                         = ('1')
ptype      : XShortEnumField                     = ('2048')
hwlen      : FieldLenField                       = ('None')
plen       : FieldLenField                       = ('None')
op         : ShortEnumField                      = ('1')
hwsrc      : MultipleTypeField (SourceMACField, StrFixedLenField) = ('None')
psrc       : MultipleTypeField (SourceIPField, SourceIP6Field, StrFixedLenField) = ('None')
hwdst      : MultipleTypeField (MACField, StrFixedLenField) = ('None')
pdst       : MultipleTypeField (IPField, IP6Field, StrFixedLenField) = ('None')
```

Paket lahko sedaj sestavimo na sledeč način:

```
>>> arppkt = Ether()/ARP()
>>> arppkt[ARP].hwsrc = "ac:92:32:e8:33:f5"
>>> arppkt[ARP].pdst = "192.168.8.145"
>>> arppkt[Ether].dst = "80:30:49:e5:6d:c5"
>>> arppkt
<Ether  dst=80:30:49:e5:6d:c5 type=ARP |<ARP  hwsrc=ac:92:32:e8:33:f5 pdst=192.168.8.145 |>>
```

Isto lahko dosežemo v eni vrstici, kjer vrednosti vsake plasti podamo kot argumente:

```
>>> arppkt = Ether(dst='80:30:49:e5:6d:c5')/ARP(hwsrc='ac:92:32:e8:33:f5', pdst='192.168.8.145')
>>> arppkt
<Ether  dst=80:30:49:e5:6d:c5 type=ARP |<ARP  hwsrc=ac:92:32:e8:33:f5 pdst=192.168.8.145 |>>
```

Layer-s se gradi enega na drugemu z uporabo **/** operatorja. Če tekom grajenja paketka pozabimo na plast, jo lahko kasneje preprosto dodamo:

```
>>> tcppkt = Ether()/IP()
>>> tcppkt
<Ether  type=IPv4 |<IP  |>>
>>> tcppkt = tcppkt/TCP()
>>> tcppkt
<Ether  type=IPv4 |<IP  frag=0 proto=tcp |<TCP  |>>>
```

### Pošiljanje paketka

Sedaj pošljimo naš zgrajen ARP paket. Ker je ARP L2 protokol bomo uporabili **sendp()** metodo. Za IP, IPv6 bi uporabili **send()** metodo.

```
>>> arppkt
<Ether  dst=80:30:49:e5:6d:c5 type=ARP |<ARP  hwsrc=ac:92:32:e8:33:f5 pdst=192.168.8.145 |>>
>>> sendp(arppkt)
.
Sent 1 packets.
>>> 
```

Glavni parametri **send** in **sendp** metode so:
* **iface** - interface iz katerega se paketki pošljejo
* **inter** - koliko sekund naj mine med pošiljanjem dveh paketkov
* **loop** - paketki se pošiljajo neskončno, če vrednost ni 0
* **pkts** - lahko je en paketek, lahko je list večih paketkov

## Ping

Kadar pričakujemo odgovor se uporablja **sr()** oziroma **srp()** metoda (send/recieve). *P* ponovno pomeni, da pošiljamo na L2, če ne na L3. Dodatno imamo metodi **sr1()** in **srp1()**, ki končata s poslušanjem potem, ko prejmeta 1 response. Drugi metodu konstantno poslušata za reply.

V primeru ARP smo sami definirali dst in src, vendar pa lahko to prepustimo Scapy-ju. Zgradili bomo le IP in ICMP layer.

```
>>> pingpkt = IP(dst="8.8.8.8")/ICMP()
>>> pingpkt.show()
###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = 192.168.8.145
  dst       = 8.8.8.8
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x0
     seq       = 0x0
     unused    = ''

>>> 
```

Za začetek bomo vzeli metodo **sr1()** ker želimo le en reply.

```
>>> response = sr1(pingpkt)
Begin emission:
Finished sending 1 packets.
..*
Received 3 packets, got 1 answers, remaining 0 packets
>>> response.show()
###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 28
  id        = 0
  flags     = 
  frag      = 0
  ttl       = 112
  proto     = icmp
  chksum    = 0x7198
  src       = 8.8.8.8
  dst       = 192.168.8.145
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0x0
     id        = 0x0
     seq       = 0x0
     unused    = ''
```

*Recieved 3 packets, gor 1 answer,...* nam samo pove koliko paketko je prejel predno je dobil naš iskani odgovor.

Še dve funkciji povezani s pošiljanjem in prejemanjem paketkov sta **srloop()** in **srploop()**. Metoda pošlje paketek in ga ponovno pošlje vsakič, ko prejme response.

Z **count** parametrom lahko definiramo kolikokrat naj se paket pošlje.

```
>>> responses = srloop(pingpkt, count=5)
RECV 1: IP / ICMP 8.8.8.8 > 192.168.8.145 echo-reply 0
RECV 1: IP / ICMP 8.8.8.8 > 192.168.8.145 echo-reply 0
RECV 1: IP / ICMP 8.8.8.8 > 192.168.8.145 echo-reply 0
RECV 1: IP / ICMP 8.8.8.8 > 192.168.8.145 echo-reply 0
RECV 1: IP / ICMP 8.8.8.8 > 192.168.8.145 echo-reply 0
        
Sent 5 packets, received 5 packets. 100.0% hits.
>>> responses
(<Results: TCP:0 UDP:0 ICMP:5 Other:0>,
 <PacketList: TCP:0 UDP:0 ICMP:0 Other:0>)
```

# Scapy in Python script

> Če so problemi glede permissions in "NoModule scapy"
```
$ sudo -E env PATH=$PATH PYTHONPATH=$PYTHONPATH python3 01.py
```

Ponovimo naš zgornji PING primer:

In [1]:
#from scapy.all import *
from scapy.all import sr1, IP, ICMP

print(sr1(IP(dst="8.8.8.8")/ICMP()).summary())

/home/balki/.local/lib/python3.10/site-packages/scapy/layers/ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/home/balki/.local/lib/python3.10/site-packages/scapy/layers/ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


PermissionError: [Errno 1] Operation not permitted

Napišimo sedaj skripto, ki nam izpisuje ARP promet.

In [ ]:
from scapy.all import ARP, sniff

def arp_display(pkt):
    if pkt[ARP].op == 1:  # who-has (request)
        return f"Request: {pkt[ARP].psrc} is asking about {pkt[ARP].pdst}"
    if pkt[ARP].op == 2:  # is-at (response)
        return f"*Response: {pkt[ARP].hwsrc} has address {pkt[ARP].psrc}"

sniff(prn=arp_display, filter="arp", store=0, count=10)

# Network Scanner

LINK - https://www.thepythoncode.com/article/building-network-scanner-using-scapy

Napisali bomo skripto, ki pregleda naprave povezane v naš network. Za naš network scanner bomo uporabili ARP request, ki bo vprašal *kdo je lastnik IP 192.168.1.10* in lastnik bo avtomatično odgovoril.

TARGET_IP najdeš na ubuntu pod Settings -> WiFi -> Settings od wifija na kerga si povezan.

In [1]:
from scapy.all import ARP, Ether, srp

target_ip = "10.1.8.1/24" #"192.168.8.1/24"
arp = ARP(pdst=target_ip)

ether = Ether(dst="ff:ff:ff:ff:ff:ff") # ff:ff:ff:ff:ff:ff MAC address indicates broadcasting

packet = ether/arp

result = srp(packet, timeout=3)[0] # index 0 are answeres, index 1 are unanswerd packets

print(result)

/home/balki/.local/lib/python3.10/site-packages/scapy/layers/ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/home/balki/.local/lib/python3.10/site-packages/scapy/layers/ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


PermissionError: [Errno 1] Operation not permitted

> In case you are not familiar with the notation "/24" or "/16" after the IP address, it is basically an IP range here, for example, "192.168.1.1/24" is a range from "192.168.1.0" to "192.168.1.255",

Rezultate imamo shranjene kot tuple `(sent_packet, received_packet)`. S for loopom bomo iteriral čez njih in izpisali relevantne informacije.

In [ ]:
from scapy.all import ARP, Ether, srp

target_ip = "192.168.8.1/24"
arp = ARP(pdst=target_ip)

ether = Ether(dst="ff:ff:ff:ff:ff:ff") # ff:ff:ff:ff:ff:ff MAC address indicates broadcasting

packet = ether/arp

result = srp(packet, timeout=3)[0] # index 0 are answeres, index 1 are unanswerd packets

print(result)

print("Devices on the network:")
print("IP \t\t\t\t MAC")
for sent, received in result:
    print(f"{received.psrc} \t\t\t {received.hwsrc}")

# Branje/Pisanje iz datoteke

Če želimo analizirati promet zapisan v datoteki lahko to naredimo na sledeč način:

In [ ]:
from scapy.all import *

# rdpcap comes from scapy and loads in our pcap file
packets = rdpcap('mycapture.cap')

# Let's iterate through every packet
for packet in packets:
    print(packet.summary())

Da zapišemo nekaj v datoteko:

In [ ]:
from scapy.all import *

# rdpcap comes from scapy and loads in our pcap file
packets = rdpcap('mycapture.cap')

# Let's iterate through every packet
for packet in packets:
    print(packet.summary())

    wrpcap("moj_capture.pcap", packet, append=True)



print("Pregled")
packets = rdpcap('moj_capture.pcap')

# Let's iterate through every packet
for packet in packets:
    print(packet.summary())